In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score

In [2]:
from tensorflow.keras.models import load_model

# Load model
model = load_model('Models/ClassifyLocations.h5') 

In [3]:
def getPredictions(videoNr, xsplit, ysplit):
    """ Gets predictions from the model
    Args:
      videoNr: Number of the video to get the predictions for 
      xsplit: Number of discrete grid splits in x direction
      ysplit: Number of discrete gird splits in y direction
      
    Returns:
      Predicted locations for all image frames in the given video
    """
    
    # Load and normalize original images
    images = np.load('ImageFrames/Images_Video' + str(videoNr) + '_10.npy') 
    images = images/images.max() 
    
    # Load and normalize dilated and thresholded images
    imagesDT = np.load('ImageFrames/Images_DilatedThresholded_Video' + str(videoNr) + '_20.npy') 
    imagesDT = imagesDT/imagesDT.max() 
    
    # Load and normalize annotations for last ten locations
    annotations = np.load('AnnotationFiles/Annotations_LastTenLocations_Video' + str(videoNr) + '_' + str(xsplit) +'_' + str(ysplit) +'_10.npy')
    annotations = annotations/annotations.max()
    
    # Get prediction from model and choose the location with highest probability
    predictions = model.predict([images, imagesDT[..., np.newaxis], annotations])
    predictedLocations = np.argmax(predictions, axis=-1)
    
    return predictedLocations

In [4]:
def calculateMAE(predictions, annotations, xsplit, ysplit):  
    """ Calculates the mean absolute error between the position predicted by the model and the true position of the ball
    Args:
      predictions: Predicted locations for all image frames 
      annotations: True locations for all image frames
      xsplit: Number of discrete grid splits in x direction
      ysplit: Number of discrete gird splits in y direction
      
    Returns:
      Mean absolute error
    """
    
    sumError = 0
    xmax = 1920
    ymax = 1088
    for i in range(len(predictions)):
        prediction = predictions[i]
        
        # Calculate predicted position of the ball (x,y) as middle point of the grid section that was predicted, if a ball was predicted
        if prediction!=0:
            xstart = int(xmax/xsplit*((prediction-1)%xsplit))
            xend = int(xstart + xmax/xsplit)
            ystart = int(ymax/ysplit*np.floor((prediction-1)/xsplit))
            yend = int(ystart + ymax/ysplit)
            x = xstart+(xend-xstart)/2
            y = ystart+(yend-ystart)/2
            predictedOutput = np.array([x, y])
            
        # Sets predicted position to (0,0), if no ball was predicted in an image frame
        else:
            predictedOutput = np.array([0, 0])
            
        # Set true position to (0,0), if no ball is in the image frame, else set postion 
        if -1 in annotations[i]:
            trueOutput = np.array([0, 0])
        else:
            trueOutput = annotations[i]

        # Calcuate euclidean distance between predicted and true position and add to totol error  
        error = np.linalg.norm(predictedOutput-trueOutput)
        sumError = sumError + error
        
    return sumError/len(predictions)

In [5]:
def evaluateVideos(videoNrs, xsplit, ysplit, printSingleVideos):  
    """ Evaluates MAE, Precision, Recall and F1-Score for given videos
    Args:
      videosNrs: Numbers of the videos to evaluate
      xsplit: Number of discrete grid splits in x direction
      ysplit: Number of discrete gird splits in y direction
      printSingleVideos: Flag that defines if stats for all single Videos should be printed
      
    Returns:
      Total Mean absolute error
      Total Precision
      Total Recall
      Total F1-Score
    """
    
    # Get Predictions and annotations for all videos and calculate the individual MAEs for each videos
    allPredictions = []
    allAnnotationsRegression = []
    allAnnotationsClassification = []
    allMae = []
    for videoNr in videoNrs:
        predictions = getPredictions(videoNr,xsplit,ysplit)
        allPredictions.append(predictions)
        annotationsRegression = np.load('AnnotationFiles/Annotations_BallPosition_Video' + str(videoNr) + '.npy')
        allAnnotationsRegression.append(allAnnotationsRegression)
        mae = calculateMAE(predictions, annotationsRegression, xsplit, ysplit)
        allMae.append(mae)
        annotationsClassification = np.load('AnnotationFiles/Annotations_ClassifyLocations_Video' + str(videoNr) + '_' + str(xsplit) +'_' + str(ysplit) +'_10.npy')
        allAnnotationsClassification.append(annotationsClassification)
        accuracy = accuracy_score(annotationsClassification, predictions)
        if(printSingleVideos):
            precision, recall, f1, _ = precision_recall_fscore_support(annotationsClassification, predictions, average='weighted')
            print('videoNr', videoNr)
            print('mae', mae)
            print('accuracy', accuracy)
            print('precision', precision)
            print('recall', recall)
            print('f1-score', f1)

    # Merge the predictions and annotations for all videos to calculate stats for all videos 
    annotationsPredictions = np.zeros((18000,1), dtype=np.uint32)
    annotationsRegressionAllVideos = np.zeros((18000,2))
    annotationsClassifyLocAllVideos = np.zeros((18000,1), dtype=np.uint32)
    maeAllVideos = 0
    shape=0
    sumAccuracy = 0
    for videoNr in videoNrs:
        annosClass = allAnnotationsClassification[videoNr-1]
        annotationsClassifyLocAllVideos[shape:shape+annosClass.shape[0]] = annosClass
        annotationsRegressionAllVideos[shape:shape+annosClass.shape[0]] = annotationsRegressionAllVideos[videoNr-1]
        annotationsPredictions[shape:shape+annosClass.shape[0]] = allPredictions[videoNr-1][...,np.newaxis]
        accuracy = accuracy_score(allAnnotationsClassification[videoNr-1], allPredictions[videoNr-1])
        sumAccuracy = sumAccuracy + accuracy
        maeAllVideos = maeAllVideos + allMae[videoNr-1]
        shape = shape + annosClass.shape[0]
    annotationsClassifyLocAllVideos = annotationsClassifyLocAllVideos[0:shape]
    annotationsRegressionAllVideos = annotationsRegressionAllVideos[0:shape]
    annotationsPredictions = annotationsPredictions[0:shape]
    precision, recall, f1, _ = precision_recall_fscore_support(annotationsClassifyLocAllVideos, annotationsPredictions, average='weighted')
    print('Total')
    print('mae', maeAllVideos/len(videoNrs))
    print('accuracy', sumAccuracy/len(videoNrs))
    print('precision', precision)
    print('recall', recall)
    print('f1-score', f1)
    
    return maeAllVideos/len(videoNrs), sumAccuracy/len(videoNrs), precision, recall, f1

In [6]:
import warnings
warnings.filterwarnings("ignore")

videoNrs = [1,2,3,4,5,6]
xsplit = 45
ysplit = 25
printSingleVideos = True

# Get mae, accuracy, precision, recall and F1-score for given videos
mae, accuracy, precision, recall, f1 = evaluateVideos(videoNrs, xsplit, ysplit, printSingleVideos)

videoNr 1
mae 22.43132783212297
accuracy 0.9419613075383589
precision 0.9315600328618071
recall 0.9419613075383589
f1-score 0.9307102661362081
videoNr 2
mae 19.212608074613897
accuracy 0.9496330887258172
precision 0.9512585773084546
recall 0.9496330887258172
f1-score 0.945367259172423
videoNr 3
mae 27.823810168775434
accuracy 0.922974324774925
precision 0.9322564969839043
recall 0.922974324774925
f1-score 0.919946405939775
videoNr 4
mae 21.090738430553102
accuracy 0.9213071023674558
precision 0.9180632598331318
recall 0.9213071023674558
f1-score 0.9129561816212518
videoNr 5
mae 14.775912217404255
accuracy 0.9789929976658887
precision 0.9808043578252014
recall 0.9789929976658887
f1-score 0.9778143776658259
videoNr 6
mae 10.918877500743971
accuracy 0.9823333333333333
precision 0.9790990513116647
recall 0.9823333333333333
f1-score 0.9791172173324779
Total
mae 19.375545704035606
accuracy 0.9495336924009631
precision 0.9530054370986317
recall 0.9495359306396932
f1-score 0.9470787606653434
